In [7]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.datasets import cifar10
import cv2
import os 
import PIL.Image

# scale an array of images to a new size
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
    # resize with nearest neighbor interpolation
        new_image = cv2.resize(image,(int(75),int(75)))
        # store
        images_list.append(new_image)
    return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
    # calculate activations
    act1 = model.predict(images1)
    act2 = model.predict(images2)
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = numpy.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


In [8]:
path1 = r"../datasets/shapes3d"
path2 = r"../hologan-2.0/generated_data/shapes3d"
size = 75

images1 = []
images2 = []
count = 0
for filename in os.listdir(path2):
    if filename.endswith('.jpg'):
      img = cv2.imread(path2+'/'+filename)
      images1.append(img)

for filename in os.listdir(path1):
    if filename.endswith('.jpg'):
        img = cv2.imread(path1+'/'+filename)
        images2.append(img)
        count = count + 1
        if count > 9:
          break

images1 = numpy.array(images1)
images2 = numpy.array(images2)
print('Loaded', images1.shape, images2.shape)

Loaded (10, 64, 64, 3) (10, 64, 64, 3)


In [9]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(size,size,3))

In [10]:
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (size,size,3))
images2 = scale_images(images2, (size,size,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)


Scaled (10, 75, 75, 3) (10, 75, 75, 3)


In [11]:
# calculate fid
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

FID: 1206.530
